## Convolutional Neural Network

In this post we will try to create a CNN for MNIST image classification using tensorflow. Lets first import the necessary libraries and import the mnist dataset.

In [19]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


Next, lets define the structure of our network. MNIST dataset have images of size 28 X 28 X 1 so that will be our input size and we will have one of the 10 classes assign to each image as image label. Note that in the dataset we have read the labels are in one hot encoding, which means that the labels would be 10 dimensional vectors.


In [22]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, 10])

Now lets start with an initial strcuture of our network, we will have 3 convolutional layers with relu activation and have a fully connected layer after that leading to output. 

So the strcture would look something like this :

input -> convolution -> relu -> maxpool -> convolution -> relu - > maxpool -> convolution -> relu -> maxpool -> fully_connected -> fully_connected -> output

Now lets create functions to create these layers. First we will create a function to create a layer to create a convolution layer with relu activation


In [33]:
def conv2d(x, weights, bias, stride = 1):
    conv_layer = tf.nn.conv2d(x, weights, strides = [1, stride, stride, 1], padding = "SAME")
    conv_layer = tf.nn.bias_add(conv_layer, bias)
    return tf.nn.relu(conv_layer)

In [35]:
def max_pool(x, k=2):
    return tf.nn.max_pool(x, ksize = [1,k,k,1], strides = [1,k,k,1], padding = "SAME")

Now lets create a function which can actually use these functions to create a CNN

In [36]:
def conv_net(x, weights, biases, dropout):
    
    #Layer 1, in this we will take input a 28 X 28 X 1 image and output a 14 X 14 X 32 layer
    
    #weights here decide the size of the filter and stride is 1 so the output of this statement would be a layer of \
    # size 28 X 28 X 32
    network = conv2d(x, weights[0], biases[0])
    
    #max pool would reduce the width and height to half as we will use k = 2
    network = max_pool(network, k = 2)

    
    
    # Layer 2 Input would be of size 14 X 14 X 32 and the output would be 7 X 7 X 64
    
    #This would result in a output of size 14 X 14 X 64 as stride = 1
    network = conv2d(network, weights[1], biases[1])
    
    # This would result in a output of size 7 X 7 X 64 as k =2 
    
    network = max_pool(network, k =2)
    
    
    
    # Fully connected layer input would be 7 X 7 X 64 and output would be 1024 nodes
    
    network = tf.reshape(network, [-1, weights[2].get_shape().as_list()[0]])
    network = tf.add(tf.matmul(network, weights[2]), biases[2])
    network = tf.nn.relu(network)
    network = tf.nn.dropout(network, dropout)
    
    
    # output layer map 1024 nodes to 10 nodes
    
    network = tf.add(tf.matmul(network, weights[3]), biases[3])
    
    
    return network
    

now lets define weights and biases

In [37]:
weights = [tf.Variable( tf.truncated_normal([5,5,1, 32])),
           tf.Variable( tf.truncated_normal([5,5,32,64])),
           tf.Variable( tf.truncated_normal([7*7*64, 1024])),
           tf.Variable( tf.truncated_normal([1024,10]))]

biases = [tf.Variable( tf.truncated_normal([32])),
          tf.Variable( tf.truncated_normal([64])),
          tf.Variable( tf.truncated_normal([1024])),
          tf.Variable( tf.truncated_normal([10]))
         ]


Now lets call this network in the session and start training the model

In [52]:
# avoids overfitting
dropout_value = tf.placeholder(tf.float32)
learning_rate = 0.001
epochs = 10
batch_size = 128
test_valid_size = 256


logits = conv_net(x, weights, biases, dropout_value)


# Cost and optimizer

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)


# Accuracy

correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            sess.run(optimizer, feed_dict ={dropout_value:0.76, x: batch_x, y: batch_y})
            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                dropout_value: 1.0})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                dropout_value: 1.0})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        dropout_value: 1.0})
    print('Testing Accuracy: {}'.format(test_acc))
    

Epoch  1, Batch   1 -Loss: 509929.8438 Validation Accuracy: 0.132812
Epoch  1, Batch   2 -Loss: 119169.1094 Validation Accuracy: 0.191406
Epoch  1, Batch   3 -Loss: 32709.6953 Validation Accuracy: 0.105469
Epoch  1, Batch   4 -Loss:  3388.8440 Validation Accuracy: 0.078125
Epoch  1, Batch   5 -Loss:  1921.1776 Validation Accuracy: 0.093750
Epoch  1, Batch   6 -Loss:  1230.7310 Validation Accuracy: 0.066406
Epoch  1, Batch   7 -Loss:   943.5535 Validation Accuracy: 0.085938
Epoch  1, Batch   8 -Loss:   776.3270 Validation Accuracy: 0.066406
Epoch  1, Batch   9 -Loss:   546.3698 Validation Accuracy: 0.093750
Epoch  1, Batch  10 -Loss:   428.3049 Validation Accuracy: 0.066406
Epoch  1, Batch  11 -Loss:   426.4393 Validation Accuracy: 0.117188
Epoch  1, Batch  12 -Loss:   242.1594 Validation Accuracy: 0.066406
Epoch  1, Batch  13 -Loss:   163.9784 Validation Accuracy: 0.117188
Epoch  1, Batch  14 -Loss:   106.1880 Validation Accuracy: 0.117188
Epoch  1, Batch  15 -Loss:   110.4948 Validati